In [ ]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

In [ ]:
# Third party packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy as sa

from dateutil.parser import parse as parse_dt

# Greg note: also adding plotly for interactive visualizations
import plotly
import plotly.express as px
import missingno

plotly.offline.init_notebook_mode()

In [ ]:
# import our packages
%reload_ext autoreload
%autoreload 2

import src.data_cleaning as data_cleaning

In [ ]:
year = 2019

# 1. Compare EIA-923 to EIA-930
EIA-923 is our source of data for any generator heat input (and thus emissions) that are not reported to CEMS, and EIA-930 is our source of data for the hourly net generation profile for each BA. Our hope is that both 923 and 930 represent the complete set of generators in each BA, and thus the net generation reported by each should be the same. Our first step is to compare the consistency of these two data sources. 

Potential sources of discrepency:
- bad data reported in EIA-930. To fix this we can try using the code from https://github.com/jdechalendar/gridemissions
- Inconsistent assignment of fuel type
- Inconsistent assignment to BA (try aggregating data at national level - it should be the same)

### Load the EIA-930 data

In [ ]:
# load EIA-930 data
# there is also an "unknown fuel sources" column but it appears to be unused
cols_to_use = ['Balancing Authority','Data Date', 'UTC Time at End of Hour',
       'Net Generation (MW) from Coal', 'Net Generation (MW) from Natural Gas',
       'Net Generation (MW) from Nuclear',
       'Net Generation (MW) from All Petroleum Products',
       'Net Generation (MW) from Hydropower and Pumped Storage',
       'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind',
       'Net Generation (MW) from Other Fuel Sources']


eia_930 = pd.concat([pd.read_csv(f'../data/downloads/eia930/EIA930_BALANCE_{year}_Jan_Jun.csv', thousands=',', usecols=cols_to_use, parse_dates=['UTC Time at End of Hour']),
                     pd.read_csv(f'../data/downloads/eia930/EIA930_BALANCE_{year}_Jul_Dec.csv', thousands=',', usecols=cols_to_use, parse_dates=['UTC Time at End of Hour'])])

# convert from end of hour timestamp to beginning of hour timestamp
eia_930['UTC Time at End of Hour'] = eia_930['UTC Time at End of Hour'] - pd.Timedelta(hours=1)

# localize the timezone as UTC time and rename the column
eia_930['UTC Time at End of Hour'] = eia_930['UTC Time at End of Hour'].dt.tz_localize('UTC')
eia_930 = eia_930.rename(columns={'UTC Time at End of Hour':'operating_datetime_utc'})

# create a column for data month
eia_930['data_month'] = eia_930['Data Date'].str.split('/', expand=True)[0]

# shorten the column name to just be the name of the fuel type
eia_930.columns = [col.replace("Net Generation (MW) from ","") for col in eia_930.columns]

eia_930.head(3)

In [ ]:
# EIA-930 also includes categories for other and unknown. These are being included in the emitting fuel source category
# TODO: check if fuel types always sum to net generation.

net_gen_columns = ['Coal', 'Natural Gas','Nuclear',
       'All Petroleum Products','Hydropower and Pumped Storage',
       'Solar', 'Wind','Other Fuel Sources']

emitting_gen_columns = ['Coal', 'Natural Gas',
       'All Petroleum Products', 'Other Fuel Sources']

In [ ]:
# Explore some data cleaning

# replace negative values with NaN
for col in emitting_gen_columns:
    eia_930.loc[eia_930[col]<0, col] = np.nan

In [ ]:
# Calculate hourly EIA-930 non-renewable generation 
eia_930["emitting_net_generation"] = eia_930[emitting_gen_columns].sum(axis=1)

# Calculate hourly EIA-930 non-renewable generation 
eia_930["total_net_generation"] = eia_930[net_gen_columns].sum(axis=1)

eia_930.head(3)

In [ ]:
annual_930 = pd.DataFrame(eia_930.groupby('Balancing Authority').sum()['emitting_net_generation'])
annual_930 = annual_930.rename(columns={'emitting_net_generation':'EIA-930'})
annual_930.head()

### Load the EIA-923 data
PUDL reports 923 monthly generation data across two tables, generation_eia923 and generation_fuel_eia923.  
PUDL says that this only contains ~55% of reported generation (https://catalystcoop-pudl.readthedocs.io/en/latest/data_sources/eia923.html).  
PUDL documentation identifies generation_fuel_eia923 as more complete. We will use this data table for now, but eventually we want to clean this data and ensure consistency.

In [ ]:
# specify the relative path to the sqllite database, and create an sqalchemy engine
pudl_db = 'sqlite:///../data/downloads/pudl/pudl_data/sqlite/pudl.sqlite'
pudl_engine = sa.create_engine(pudl_db)

# load fuel table, filter for year. Table is large so filter before loading into pandas
gen_fuel_923 = pd.read_sql(f"SELECT * FROM generation_fuel_eia923 \
                           WHERE report_date >= '{year}-01-01' \
                           AND report_date <= '{year}-12-30'", pudl_engine)
gen_fuel_923.head()

In [ ]:
# identify the fuel code to fuel category mapping that pudl performed
fuel_code_dict_pudl = gen_fuel_923.drop_duplicates(['energy_source_code','fuel_type_code_pudl'])
fuel_code_dict_pudl = pd.Series(fuel_code_dict_pudl.fuel_type_code_pudl.values, index=fuel_code_dict_pudl.energy_source_code).to_dict()


In [ ]:
# modify pudl's mapping to match EIA-930 fuel types
fuel_code_dict_930 = {'DFO': 'oil',
 'WND': 'wind',
 'WAT': 'hydro',
 'NG': 'natgas', # changed natural gas from gas to natgas
 'BIT': 'coal',
 'SUB': 'coal',
 'LIG': 'coal',
 'PG': 'other', # changed process gas from gas to other
 'RC': 'coal',
 'AB': 'other', # changed agricultural byproduct from waste to other
 'WDS': 'other', # changed from waste to other
 'RFO': 'oil',
 'LFG': 'other', # changed landfill gas from waste to other
 'PC': 'coal',
 'SUN': 'solar',
 'OBG': 'other', #changed other biobgas from waste to other
 'GEO': 'other', # geothermal
 'MWH': 'other', # batteries / energy storage - there is a change this also includes pumped storage hydro
 'OG': 'other', # changed other gas from gas to other
 'WO': 'oil',
 'JF': 'oil',
 'KER': 'oil',
 'OTH': 'other',
 'WC': 'coal',
 'SGC': 'other', # changed from gas to other
 'OBS': 'other', # changed from waste to other
 'TDF': 'other', # changed from waste to other
 'BFG': 'other', # changed from gas to other
 'MSB': 'other', # changed from waste to other
 'MSN': 'other', # changed from waste to other
 'SC': 'coal',
 'BLQ': 'other', # changed from waste to other
 'WH': 'other',
 'OBL': 'other', # changed from waste to other
 'SLW': 'other', # changed from waste to other
 'PUR': 'other', 
 'WDL': 'other', # changed from waste to other
 'SGP': 'other'} # changed from gas to other

# add a column for 930 fuel type
gen_fuel_923['fuel_type_930'] = gen_fuel_923['energy_source_code'].map(fuel_code_dict_930)

# Filter for emitting generation only 
gen_fuel_923 = gen_fuel_923[gen_fuel_923.fuel_type_930.isin(['oil', 'natgas', 'coal', 'other'])]

# load the plants_entity_eia data
plants_ba = pd.read_sql("SELECT balancing_authority_code_eia, plant_id_eia \
                            FROM plants_entity_eia", pudl_engine)

# Add ba code to generation_fuel_eia923
gen_fuel_923 = gen_fuel_923.merge(plants_ba, how='left', on='plant_id_eia')

gen_fuel_923.head()

In [ ]:
annual_923 = pd.DataFrame(gen_fuel_923.groupby("balancing_authority_code_eia").sum()["net_generation_mwh"])
annual_923 = annual_923.rename(columns={'net_generation_mwh':'EIA-923'})
annual_923.head()

In [ ]:
compare_annual_923_930 = annual_923.merge(annual_930, how='outer', left_index=True, right_index=True)

In [ ]:
# identify which BAs are missing from one or another dataset
compare_annual_923_930[compare_annual_923_930.isna().any(axis=1)]

In [ ]:
# let's compare each BA
px.scatter(compare_annual_923_930)

In [ ]:
percent_diff_923_930 =(compare_annual_923_930['EIA-930'] - compare_annual_923_930['EIA-923']) / compare_annual_923_930['EIA-923'] * 100

px.bar(percent_diff_923_930, title='percent difference between emitting net generation in EIA-930 and EIA-923', labels={'value':'% change from EIA-923'})


### Let's explore these differences by fuel type

In [ ]:
annual_923_by_fuel = gen_fuel_923.groupby(["balancing_authority_code_eia","fuel_type_930"]).sum()["net_generation_mwh"].reset_index()
annual_923_by_fuel = annual_923_by_fuel.rename(columns={'net_generation_mwh':'EIA-923'})
annual_923_by_fuel.head(3)

In [ ]:
annual_930_by_fuel = eia_930.groupby('Balancing Authority').sum()[['Coal','Natural Gas','All Petroleum Products', 'Other Fuel Sources']]
annual_930_by_fuel = annual_930_by_fuel.rename(columns={'Coal':'coal','Natural Gas':'natgas','All Petroleum Products':'oil', 'Other Fuel Sources':'other'})
annual_930_by_fuel = annual_930_by_fuel.reset_index().melt(id_vars='Balancing Authority', var_name='fuel_type_930', value_name='EIA-930')
annual_930_by_fuel = annual_930_by_fuel.rename(columns={'Balancing Authority':'balancing_authority_code_eia'})
annual_930_by_fuel

In [ ]:
compare_annual_by_fuel = annual_923_by_fuel.merge(annual_930_by_fuel, 
                                                  how='outer', 
                                                  on=['balancing_authority_code_eia','fuel_type_930'])

px.scatter(compare_annual_by_fuel, 
           x='balancing_authority_code_eia', 
           y=['EIA-923','EIA-930'], 
           facet_col='fuel_type_930', 
           facet_col_wrap=1, 
           height=1000,
           title='comparison of net generation by fuel type for each BA')

In [ ]:
percent_error = compare_annual_by_fuel.groupby('fuel_type_930').sum()
# add a total row
total = pd.DataFrame(percent_error.sum(), columns=['total']).T
percent_error = pd.concat([percent_error, total], axis=0)
percent_error['percent_error'] = (percent_error['EIA-930'] - percent_error['EIA-923']) / percent_error['EIA-923']
percent_error

### Percent Error results
Initial run with no data cleaning: EI-930 percent diff from EIA-923
 - coal:    -2.67%
 - natgas:  -8.38%
 - oil:     782.0%
 - other:   -28.77%
 - total:   -5.96%

After replacing all negative values in EIA-930 with NaNs: (this seems to have slightly reduced errors)
 - coal:    -2.64%
 - natgas:  -8.22%
 - oil:     782.3%
 - other:   -28.68%
 - total:   -5.85%

# 1. Compare hourly generation to EIA-930
* Compare emisson rate
* Compare total emission 

In [ ]:
hourly_emission_rate = pd.read_csv("../data/outputs/hourly_emission_rate.csv", index_col=0, parse_dates=True)
hourly_net_generation = pd.read_csv("../data/outputs/hourly_net_generation.csv", index_col=0, parse_dates=True)
hourly_net_emissions = pd.read_csv("../data/outputs/hourly_net_emission.csv", index_col=0, parse_dates=True)
hourly_emission_rate.head()

In [ ]:
eia_930.head()

In [ ]:
# large BAs with significant discrepancies between eGRID and aggregated CEMS data: PJM, MISO, ERCO
# BAs with bad 930 data: PGE, AVRN, BPAT, "FPL","IPCO","NEVP","SC","TEC","TVA"
ba = "AVRN"

fig, (ax, ax2) = plt.subplots(1,2, figsize=(14,5))

ax.plot(hourly_net_generation[ba], label="BA-aggregated CEMS")
ax.plot(eia_930[eia_930["Balancing Authority"]==ba]["operating_datetime_utc"],\
        eia_930[eia_930["Balancing Authority"]==ba]["emitting_net_generation"])
ax.set_title(ba)

ax2.plot(hourly_net_generation[ba], label="Hourly eGRID")
ax2.plot(eia_930[eia_930["Balancing Authority"]==ba]["operating_datetime_utc"],\
         eia_930[eia_930["Balancing Authority"]==ba]["emitting_net_generation"], label="EIA-930 emitting")
ax2.set_title(ba)
ax2.legend()
ax2.set_xlim(parse_dt("2019-08-01"), parse_dt("2019-08-10"))

In [ ]:
ba = "PJM"

data_for_plot = eia_930.loc[eia_930["Balancing Authority"]==ba, ["930_generation_non_renewable","930_generation_minus_renewable"]]
data_for_plot = data_for_plot.merge(hourly_net_generation[ba], how='left', left_index=True, right_index=True)
data_for_plot = data_for_plot.rename(columns={ba:'CEMS'})

px.line(data_for_plot, title=f'Net generation in {ba} EIA-930 vs CEMS')

### Annual EIA-930

Hourly EIA-930 appears to be in good agreement with hourly eGRID numbers, even in areas like PJM where hourly eGRID covers only 60% of actual eGRID non-renewable generation. Is this something about aggregation, or a difference between EIA-930 and actual eGRID? 

In [ ]:
# For annual comparison graphs, see below with eGRID
annual_eia_930 = eia_930.groupby("Balancing Authority").sum()["emitting_net_generation"]
annual_eia_930.head()

# 2. Compute annual generation from 923
* Aggregate 923 generation to the BA level 
* In (3), compare aggregated 923 to aggregated hourly eGRID, annual eGRID, and 930

In [ ]:
# PUDL reports 923 monthly generation data across two tables, generation_eia923 and generation_fuel_eia923

# Load generation, filter for year. PUDL says that this only contains ~55% of reported generation 
# (https://catalystcoop-pudl.readthedocs.io/en/latest/data_sources/eia923.html)
# Table is large, so filter before load
gen_923 = pd.read_sql(f"SELECT * FROM generation_eia923 \
                           WHERE report_date >= '{year}-01-01' \
                           AND report_date <= '{year}-12-30'", pudl_engine)
gen_923.head()

In [ ]:
# Are all the plants in gen_923 also in gen_fuel_923?
oneway = np.setdiff1d(gen_923["plant_id_eia"].unique(), gen_fuel_923["plant_id_eia"].unique())
print(f"{len(oneway)} plants in generation_eia923 are not in generation_fuel_eia923")

otherway = np.setdiff1d(gen_fuel_923["plant_id_eia"].unique(),gen_923["plant_id_eia"].unique())
print(f"{len(oneway)} plants in generation_fuel_eia923 are not in generation_eia923")

Both PUDL tables appear to contain the same plants, but PUDL documentation identifies generation_fuel_eia923 as more complete. We use that table to aggregate...

# 2. Compare annually-aggregated generation to eGRID 
* Compare annual aggregated generation, emission, and emission rate

In [ ]:
egrid = pd.read_excel(f"../data/downloads/egrid/egrid{year}_data.xlsx", sheet_name=f"BA{year-2000}", header=1, index_col="BACODE")
egrid.head()

In [ ]:
## Get names (row 0 of egrid) and codes (row 1) for each eGRID data type
## We use data codes as columns (above), but mapping back to names is useful 
egrid_data_code_to_name = pd.read_excel(f"../data/downloads/egrid/egrid{year}_data.xlsx", sheet_name=f"BA{year-2000}", nrows=1)
egrid_data_code_to_name = dict([(egrid_data_code_to_name.loc[0,name], name) for name in egrid_data_code_to_name.columns])

In [ ]:
# Merge summed hourly eGRID generation and annual eGRID non-renuable net generation
annual_generation = hourly_net_generation.sum(axis=0).rename("hourly")
annual_generation = egrid.merge(annual_generation, how='right', left_index=True, right_index=True).loc[:,["hourly","BAGENACY"]]

In [ ]:
# Merge aggregated 930, 923
annual_generation = annual_generation.merge(annual_eia_930, how="left", left_index=True, right_index=True)
annual_generation = annual_generation.merge(annual_923, how="left", left_index=True, right_index=True)

In [ ]:
annual_generation.head()

In [ ]:
fig, (ax, ax1, ax2, ax3) = plt.subplots(4, 1, figsize=(12,24))

## Note: 923 and eGRID exactly agree, so points are on top of one another. 
ax.scatter(annual_generation.index, annual_generation["hourly"].tolist(), label="Aggregated hourly eGRID")
ax.scatter(annual_generation.index, annual_generation["BAGENACY"].tolist(), label="eGRID")
ax.scatter(annual_generation.index, annual_generation["930_generation_non_renewable"].tolist(), label="930 non-renewables")
ax.scatter(annual_generation.index, annual_generation["annual_923_net_gen"].tolist(), label="923")
ax.legend()
ax.grid(axis='x')
ax.set_title(f"{egrid_data_code_to_name['BAGENACY']}")
ax.set_xticklabels(labels=annual_generation.index, rotation=90);
ax.set_ylabel("MWh non-renewable generation")

small = annual_generation[annual_generation["BAGENACY"] < 1e8]
ax1.scatter(small.index, small["hourly"].tolist())
ax1.scatter(small.index, small["BAGENACY"].tolist())
ax1.scatter(small.index, small["930_generation_non_renewable"].tolist(), label="930 non-renewables")
ax1.grid(axis='x')
ax1.set_title("BAs under 100,000,000 MWh annual")
ax1.set_xticklabels(labels=small.index, rotation=90);

ax2.bar(annual_generation.index, annual_generation["hourly"]/annual_generation["BAGENACY"])
ax2.grid(axis='x')
ax2.set_xticklabels(labels=annual_generation.index, rotation=90);
ax2.hlines([1.0], -.5, len(annual_generation)-0.5, color='r')
ax2.set_ylabel("Fraction of eGRID non-renewable generation captured by CEMS");
ax2.set_title("Annual vs Hourly eGRID")

ax3.bar(annual_generation.index, annual_generation["hourly"]/annual_generation["930_generation_non_renewable"])
ax3.grid(axis='x')
ax3.set_xticklabels(labels=annual_generation.index, rotation=90);
ax3.hlines([1.0], -.5, len(annual_generation)-0.5, color='r')
ax3.set_ylabel("Fraction of 930 generation captured by CEMS");
ax3.set_ylim(0,2)
ax3.set_title("EIA-930 vs hourly eGRID")

In [ ]:
# Many small BAs have terrible coverage of EIA-930 data, resulting in low annually aggregated 930 numbers.
# The worst discrepencies between 930 and eGRID:
annual_generation.loc[["FPL","IPCO","NEVP","SC","TEC","TVA"],:]

# 5. Compare eGRID to 923

In [ ]:
fig, (ax) = plt.subplots(1, figsize=(12, 6))

ax.bar(annual_generation.index, annual_generation["annual_923_net_gen"]/annual_generation["BAGENACY"])
ax.grid(axis='x')
ax.set_xticklabels(labels=annual_generation.index, rotation=90);
ax.hlines([1.0], -.5, len(annual_generation)-0.5, color='r')
ax.set_ylabel("Fraction of eGRID non-renewable generation captured by 923");
ax.set_title("923 vs eGRID")